In [ ]:
import requests
import numpy as np
import tensorflow as tf
import time

# -------------------- CONFIG --------------------
THINGSPEAK_CHANNEL_ID = "2933580"
THINGSPEAK_READ_API_KEY = "E4NV36VDL9CYRGEB"
BLYNK_AUTH = "dG5x60LW0ivdfNbo7U2RjVZr9xpJQoMe"

HEART_RATE_PIN = 0  # Blynk virtual pin for live HR (V0)
GRAPH_PIN = 2       # Blynk virtual pin for predicted avg HR (V2)

# Load models
# predict_model = tf.keras.models.load_model("/content/CONVLSTM_30EPOCHS.h5")
model = tf.keras.models.load_model("/content/CONVLSTM_30EPOCHS.h5", compile=False)
anomaly_model = tf.keras.models.load_model("/content/ANOMALY_DETECTION.h5")

# -------------------- FUNCTION: Get last 60 HR values from ThingSpeak --------------------
def get_last_60_hr_values():
    url = f"https://api.thingspeak.com/channels/{THINGSPEAK_CHANNEL_ID}/fields/1.json?api_key={THINGSPEAK_READ_API_KEY}&results=60"
    response = requests.get(url)
    data = response.json()
    feeds = data["feeds"]
    heart_rates = []

    for item in feeds:
        try:
            val = float(item["field1"])
            heart_rates.append(val)
        except:
            continue

    if len(heart_rates) >= 5:
        return np.array(heart_rates[-5:])  # Last 60 values
    else:
        print("⚠️ Not enough HR data (need 5).")
        return None

# -------------------- FUNCTION: Send data to Blynk --------------------
def send_to_blynk(pin, value):
    requests.get(f"https://blynk.cloud/external/api/update?token={BLYNK_AUTH}&V{pin}={value}")

# -------------------- FUNCTION: Send notification to Blynk --------------------
def send_notification(msg):
    requests.get(f"https://blynk.cloud/external/api/notify?token={BLYNK_AUTH}&message={msg}")

# -------------------- FUNCTION: Predict next 60 seconds HR --------------------
# def predict_next_minute(hr_sequence):
#     # Reshape to (1, 60, 1) for LSTM/CNN input
#     input_seq = hr_sequence.reshape(1, 5, 1)
#     predicted_hr = model.predict(input_seq)
#     return predicted_hr.flatten()


def predict_next_minute(hr_sequence):
    # Reshape to (1, 1, 140, 1) for LSTM/CNN input
    # Ensure hr_sequence has 140 data points. Pad or truncate if necessary
    hr_sequence = np.pad(hr_sequence, (0, 140 - len(hr_sequence)), 'constant')[:140]
    input_seq = hr_sequence.reshape(1, 1, 140, 1)
    predicted_hr = model.predict(input_seq)
    return predicted_hr.flatten()




# -------------------- FUNCTION: Detect anomaly --------------------
# def detect_anomaly(predicted_hr):
#     # Reshape to match input shape of anomaly model
#     prediction_input = predicted_hr.reshape(1, -1)
#     result = anomaly_model.predict(prediction_input)
#     return result[0][0] > 0.5  # Customize threshold if needed

# -------------------- MAIN LOOP --------------------
while True:
    hr_data = get_last_60_hr_values()

    if hr_data is not None:
        print("📥 Got past 60 heart rate values from ThingSpeak")
        print(f"⏱️  Past HR values: {hr_data[:5]} ...")

        # Send live value to Blynk (latest reading)
        send_to_blynk(HEART_RATE_PIN, hr_data[-1])

        # Predict next 60 seconds HR
        predicted_hr = predict_next_minute(hr_data)
        avg_pred = round(np.mean(predicted_hr), 2)

        print(f"📈 Predicted next HR (next 60 sec): {predicted_hr[:5]} ...")
        print(f"📊 Predicted Avg HR: {avg_pred}")

        # Send predicted avg to Blynk
        send_to_blynk(GRAPH_PIN, avg_pred)

        # Detect anomaly
        # if detect_anomaly(predicted_hr):
        #     print("🚨 Anomaly detected!")
        #     send_notification("⚠️ Anomaly in predicted heart rate!")

    else:
        print("❌ Could not fetch enough data")

    time.sleep(30)  # Wait 30 seconds before checking again
